In [ ]:
#Python libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import numpy as np
from shapely import wkt

In [ ]:
# Load data, process geometries, create a map, color buildings by nearby services, add tooltips & legend, display map

In [ ]:
# Load the CSV file
resultados_unidos = pd.read_csv("servicos_categoria.csv")

# Convert WKT geometry to Shapely objects
resultados_unidos['geometry'] = resultados_unidos['geometry_wkt'].apply(wkt.loads)

# Create a GeoDataFrame
resultados_unidos = gpd.GeoDataFrame(resultados_unidos, geometry='geometry', crs="EPSG:4326")

# Ensure all 'numero_servicos_proximos' values are numeric and fill NaN with 0
resultados_unidos['numero_servicos_proximos'] = pd.to_numeric(resultados_unidos['numero_servicos_proximos'], errors='coerce').fillna(0)

# Reproject to a projected CRS for accurate centroid calculations
resultados_unidos_proj = resultados_unidos.to_crs(epsg=3857)

# Calculate the center of the map
centro_lat = resultados_unidos_proj.geometry.centroid.to_crs(epsg=4326).y.mean()
centro_lon = resultados_unidos_proj.geometry.centroid.to_crs(epsg=4326).x.mean()

# Create the map centered at the computed midpoint
m = folium.Map(location=[centro_lat, centro_lon], zoom_start=14)

# Determine normalization range
vmin = resultados_unidos['numero_servicos_proximos'].min()
vmax = resultados_unidos['numero_servicos_proximos'].max()
if vmin == vmax:
    vmin = 0
    vmax = vmax + 1  # Prevents division by zero

# Function to determine color based on 'numero_servicos_proximos'
def get_color(numero_servicos_proximos):
    cmap = plt.get_cmap('YlOrRd')
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    return to_hex(cmap(norm(numero_servicos_proximos))) if numero_servicos_proximos > 0 else '#d3d3d3'  # Light gray for zero values

# Create building layer
edificios_layer = folium.FeatureGroup(name='Buildings').add_to(m)

# Ensure the DataFrame contains the required column
if 'numero_servicos_proximos' not in resultados_unidos.columns:
    print("Error: Column 'numero_servicos_proximos' not found in DataFrame!")

# Add buildings to the map with correctly assigned properties
geojson = folium.GeoJson(
    resultados_unidos,
    name="Buildings",
    style_function=lambda feature: {
        'fillColor': get_color(feature['properties'].get('numero_servicos_proximos', 0)),  # Default value of 0 if not found
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['numero_servicos_proximos', 'servicos_por_categoria'],
        aliases=['Nearby services:', 'Services by category:'],
        localize=True
    )
).add_to(edificios_layer)

# Add layer control
folium.LayerControl().add_to(m)

# Create color scale
colormap = plt.get_cmap('YlOrRd')
color_scale = folium.StepColormap(
    [to_hex(colormap(i)) for i in np.linspace(0, 1, 256)],
    vmin=vmin,
    vmax=vmax,
    caption='Number of Nearby Services'
)
color_scale.add_to(m)

# Display the map
m